In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

%matplotlib inline

# For preprocessing
import csv
from os import path
from pathlib import Path

# Set the project root folder
root_dir = Path().absolute()

In [5]:
import re

In [6]:
try:
    data = pd.read_csv('../data/accre-jobs-2020.csv')
except Exception as error:
    print(error)

Error tokenizing data. C error: Expected 13 fields in line 3461, saw 15



In [7]:
try:
    # The data might have already been pre-processed previoulsy
    # If not, this line would generate a “No such file” error so we would fallback to the exception below
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    print('Reading from previously processed file...')
    display(data.head())
except FileNotFoundError:
    print('Preprocessed file not found. Creating a new one. Please wait...')
    # Open 2 files: One is the source and one is the destination
    with open(path.join(root_dir, '..', 'data', 'accre-jobs-2020.csv'), mode='r') as source,\
    open(path.join(root_dir, '..', 'data', 'accre-jobs-2020-processed.csv'), mode='w', newline='') as destination:
        # Set file reader and file writer on the source and destination
        reader = csv.reader(source)
        writer = csv.writer(destination)
        # Go through each line in the source
        for line in reader:
            # Create a new matching line with the last column within quotes
            # This will make it into a single column if there are more commas
            newline = line[:12] + [','.join(line[12:])]
            # Write to the destination file
            writer.writerow(newline)
    # Try importing as normal again on the newly created destination file
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    display(data.head())
except Error as error:
    print('Something went wrong:', error)

Reading from previously processed file...


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [8]:
accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020-processed.csv').rename(columns = str.lower)
accre_jobs_2020 = accre_jobs_2020[accre_jobs_2020.state.str.contains('RUNNING') == False]

In [9]:
accre_jobs_2020.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', error_bad_lines = False)

accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', nrows = 3459)

line 612270: expected 13 fields, saw 100\nSkipping line 612271

accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12]).rename(columns = str.lower)

In [10]:
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312


In [11]:
accre_jobs_2020['state'].value_counts()

COMPLETED              3804644
CANCELLED                 6107
FAILED                    1406
CANCELLED by 686562        681
OUT_OF_MEMORY               94
TIMEOUT                     80
CANCELLED by 505355         63
CANCELLED by 855431         17
CANCELLED by 782611         13
CANCELLED by 9206            9
CANCELLED by 397600          7
CANCELLED by 200557          6
CANCELLED by 0               5
CANCELLED by 454080          4
CANCELLED by 199766          4
CANCELLED by 486541          4
CANCELLED by 483348          3
CANCELLED by 9201            3
CANCELLED by 649319          3
CANCELLED by 9202            3
CANCELLED by 546080          2
CANCELLED by 666860          2
CANCELLED by 791651          2
CANCELLED by 90423           2
CANCELLED by 124006          2
CANCELLED by 515423          2
CANCELLED by 693461          2
CANCELLED by 895426          2
CANCELLED by 199066          2
CANCELLED by 785271          1
CANCELLED by 337422          1
CANCELLED by 781109          1
CANCELLE

In [12]:
accre_cancelled_by = accre_jobs_2020[accre_jobs_2020['state'] == 'CANCELLED by 686562']
accre_cancelled_by

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
292610,17494426,sharlin,page,10240Mn,0,7-00:00:00,00:00:14,1,1,production,0:0,CANCELLED by 686562,cn1089
292615,17494431,sharlin,page,10240Mn,0,7-00:00:00,00:00:14,1,1,production,0:0,CANCELLED by 686562,cn1128
292616,17494432,sharlin,page,10240Mn,0,7-00:00:00,00:00:06,1,1,production,0:0,CANCELLED by 686562,cn1127
292618,17494434,sharlin,page,10240Mn,0,7-00:00:00,00:00:07,1,1,production,0:0,CANCELLED by 686562,cn1122
292623,17494439,sharlin,page,10240Mn,0,7-00:00:00,00:00:06,1,1,production,0:0,CANCELLED by 686562,cn425
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309575,17522665,sharlin,page,10240Mn,0,7-00:00:00,00:00:01,1,1,production,0:0,CANCELLED by 686562,cn442
309576,17522666,sharlin,page,10240Mn,0,7-00:00:00,00:00:00,1,1,production,0:0,CANCELLED by 686562,cn1316
309578,17522668,sharlin,page,10240Mn,0,7-00:00:00,00:00:01,1,1,production,0:0,CANCELLED by 686562,cn1127
309579,17522669,sharlin,page,10240Mn,0,7-00:00:00,00:00:05,1,1,production,0:0,CANCELLED by 686562,cn1127


In [13]:
accre_jobs_2020['account'].nunique()

175

In [14]:
accre_jobs_2020['account'].value_counts()

cms          806456
cep          609282
summer       380840
galia        198036
carrot       192551
              ...  
discovery         1
purple            1
flamingo          1
hass              1
tree              1
Name: account, Length: 175, dtype: int64

In [58]:
accre_jobs_2020['user'].value_counts().shape

(597,)

In [15]:
accre_jobs_2020['nodes'].value_counts()

1      3807797
2         2540
3          885
4          604
5          417
        ...   
39           1
38           1
37           1
109          1
500          1
Name: nodes, Length: 83, dtype: int64

In [16]:
accre_jobs_2020[accre_jobs_2020.account.str.contains('cms')]

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
2125,17032244,cms,cmspilot,28000Mn,8320.07M,2-00:00:00,1-23:31:46,1,4,production,0:0,COMPLETED,cn1205
2126,17032568,cms,cmspilot,28000Mn,11954.84M,2-00:00:00,1-23:33:13,1,4,production,0:0,COMPLETED,cn1492
2127,17032569,cms,cmspilot,28000Mn,10498.84M,2-00:00:00,1-23:36:36,1,4,production,0:0,COMPLETED,cn1497
2128,17032916,cms,cmspilot,21875Mn,11728.02M,2-00:00:00,1-23:27:55,1,8,nogpfs,0:0,COMPLETED,ng1037
2129,17032920,cms,cmspilot,94451Mn,20659.08M,2-00:00:00,1-23:16:04,1,8,nogpfs,0:0,COMPLETED,ng691
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816215,24173661,cmsadmin,autocms,4096Mc,860.03M,12:00:00,00:03:50,1,1,production,0:0,COMPLETED,cn394
3816221,24173686,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:02:29,1,1,sam,0:0,COMPLETED,vm-cms-sam-pri
3816233,24173689,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:02:11,1,1,sam,0:0,COMPLETED,vm-cms-sam-sec
3816240,24173711,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:01:29,1,1,sam,0:0,COMPLETED,vm-cms-sam-pri


In [17]:
cms_jobs = accre_jobs_2020[accre_jobs_2020.account.str.contains('cms')]

In [18]:
cms_jobs['account'].value_counts()

cms            806456
cms_lowprio    135682
cmsadmin       102589
cms_gpu_acc      1196
Name: account, dtype: int64

In [19]:
cms_jobs.shape

(1045923, 13)

In [20]:
cms_jobs['nodes'].value_counts()

1    1045922
5          1
Name: nodes, dtype: int64

In [21]:
cms_Mc = cms_jobs[cms_jobs.reqmem.str.contains('Mc')]
cms_Mc

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
7865,17092491,cmsadmin,autocms,2048Mc,819.12M,12:00:00,04:00:11,1,1,production,0:0,COMPLETED,cn1134
8186,17094019,cmsadmin,autocms,2048Mc,789.33M,12:00:00,00:08:01,1,1,production,0:0,COMPLETED,cn1088
8225,17094164,cmsadmin,autocms,2048Mc,805.13M,12:00:00,00:06:19,1,1,production,0:0,COMPLETED,cn913
8251,17094286,cmsadmin,autocms,2048Mc,863.34M,12:00:00,00:03:24,1,1,production,0:0,COMPLETED,cn1081
8327,17094473,cmsadmin,autocms,2048Mc,870.35M,12:00:00,00:04:48,1,1,production,0:0,COMPLETED,cn1278
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816072,24173338,cmsadmin,autocms,4096Mc,872.41M,12:00:00,00:04:17,1,1,production,0:0,COMPLETED,cn394
3816132,24173475,cms,lailah,1024Mc,62.46M,1-20:00:00,00:00:47,1,1,production,0:0,COMPLETED,cn394
3816144,24173491,cmsadmin,autocms,4096Mc,869.54M,12:00:00,00:04:19,1,1,production,0:0,COMPLETED,cn421
3816175,24173567,cms,lailah,1024Mc,1014.77M,1-20:00:00,00:05:20,1,1,production,0:0,COMPLETED,cn394


In [22]:
#accre_jobs_2020[accre_jobs_2020["account"]].str.match('\d+').value_counts()

In [23]:
accre_jobs_2020['usedmem'].str[-1].value_counts()

M    3213712
0     599470
Name: usedmem, dtype: int64

In [24]:
accre_mem_zeros = accre_jobs_2020[(accre_jobs_2020['usedmem'] == '0') & (accre_jobs_2020['state'] == 'COMPLETED')]
accre_mem_zeros

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
4805,17072928,endive,bennett,8192Mn,0,5-00:00:00,00:00:25,1,1,turing,0:0,COMPLETED,gpu0039
4806,17072929,endive,bennett,8192Mn,0,5-00:00:00,00:00:34,1,1,turing,0:0,COMPLETED,gpu0039
4807,17072930,endive,bennett,8192Mn,0,5-00:00:00,00:00:23,1,1,turing,0:0,COMPLETED,gpu0039
4809,17072932,endive,bennett,8192Mn,0,5-00:00:00,00:01:05,1,1,turing,0:0,COMPLETED,gpu0039
4810,17072933,endive,bennett,8192Mn,0,5-00:00:00,00:00:27,1,1,turing,0:0,COMPLETED,gpu0039
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816245,24173761_4,portabella,vennie,16384Mn,0,1-00:00:00,00:00:36,1,1,production,0:0,COMPLETED,cn1135
3816258,24173800_3,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1133
3816259,24173800_4,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1092
3816260,24173800_5,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1135


# what??

In [25]:
accre_mem_zeros['account'].value_counts()

cep              143534
orange            81295
cms               74671
boysenberries     29640
bulk              29224
                  ...  
discovery             1
hass                  1
with                  1
romanesco             1
crowns                1
Name: account, Length: 148, dtype: int64

In [26]:
accre_jobs_2020['account'].value_counts()

cms          806456
cep          609282
summer       380840
galia        198036
carrot       192551
              ...  
discovery         1
purple            1
flamingo          1
hass              1
tree              1
Name: account, Length: 175, dtype: int64

In [27]:
accre_mem_zeros['partition'].value_counts()

production        537051
nogpfs             44153
pascal              5982
turing              4238
maxwell             1062
debug                686
sam                   99
cgw-dougherty1         9
Name: partition, dtype: int64

accre_mem_zeros['partition'].plot(kind = 'bar')

In [28]:
accre_jobs_2020[["exitcode_user", "exitcode_error"]] = accre_jobs_2020["exitcode"].str.split(":", expand=True)


In [29]:
accre_jobs_2020.head(3) 

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0


In [30]:
accre_jobs_2020['used_mb_per_node'] = accre_jobs_2020['usedmem'].str.strip('M').astype(float)
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75


In [31]:
accre_jobs_2020['used_mb_per_core'] = (accre_jobs_2020['used_mb_per_node'])/(accre_jobs_2020['cpus']*accre_jobs_2020['nodes'])
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49,2748.895417
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12,2799.213333
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86,2879.660833
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33,2721.555417
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11,2744.837917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08,11634.540000
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81,10439.905000
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68,15021.340000
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75,15533.875000


In [32]:
#accre_jobs_2020['reqmem_mc'] = accre_jobs_2020[accre_jobs_2020['reqmem'].str.contains('Mc')]
#accre_jobs_2020['reqmem_mn'] = accre_jobs_2020[accre_jobs_2020['reqmem'].str.contains('Mn')]
#accre_jobs_2020

In [33]:
#Maeva
condition = accre_jobs_2020["reqmem"].str.contains("Mc")
accre_jobs_2020.loc[condition, "reqmem_mc"] = accre_jobs_2020["reqmem"].str[:-2]

In [34]:
condition = accre_jobs_2020["reqmem"].str.contains("Mn")
accre_jobs_2020.loc[condition, "reqmem_mn"] = accre_jobs_2020["reqmem"].str[:-2]

In [35]:
accre_jobs_2020['reqmem_mn'] = accre_jobs_2020['reqmem_mn'].astype(float)
accre_jobs_2020['reqmem_mc'] = accre_jobs_2020['reqmem_mc'].astype(float)
accre_jobs_2020['reqmem_mn_per_core'] = (accre_jobs_2020['reqmem_mn'])/(accre_jobs_2020['cpus']*accre_jobs_2020['nodes'])

In [36]:
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn,reqmem_mn_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49,2748.895417,NaN,122880.0,5120.0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12,2799.213333,NaN,122880.0,5120.0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86,2879.660833,NaN,122880.0,5120.0
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33,2721.555417,NaN,122880.0,5120.0
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11,2744.837917,NaN,122880.0,5120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08,11634.540000,NaN,32768.0,16384.0
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81,10439.905000,NaN,32768.0,16384.0
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68,15021.340000,NaN,32768.0,16384.0
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75,15533.875000,NaN,32768.0,16384.0


In [37]:
accre_jobs_2020.rename(columns={'reqmem_per_core':'reqmem_mn_per_core'}, inplace=True)
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn,reqmem_mn_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49,2748.895417,NaN,122880.0,5120.0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12,2799.213333,NaN,122880.0,5120.0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86,2879.660833,NaN,122880.0,5120.0
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33,2721.555417,NaN,122880.0,5120.0
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11,2744.837917,NaN,122880.0,5120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08,11634.540000,NaN,32768.0,16384.0
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81,10439.905000,NaN,32768.0,16384.0
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68,15021.340000,NaN,32768.0,16384.0
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75,15533.875000,NaN,32768.0,16384.0


In [38]:
accre_jobs_2020.loc[accre_jobs_2020['reqmem_mn_per_core'].isnull(),'reqmem_mn_per_core'] = accre_jobs_2020['reqmem_mc']
accre_jobs_2020['reqmem_per_core'] = accre_jobs_2020['reqmem_mn_per_core']
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,...,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn,reqmem_mn_per_core,reqmem_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,...,COMPLETED,cn1531,0,0,65973.49,2748.895417,NaN,122880.0,5120.0,5120.0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,...,COMPLETED,cn1441,0,0,67181.12,2799.213333,NaN,122880.0,5120.0,5120.0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,...,COMPLETED,cn1464,0,0,69111.86,2879.660833,NaN,122880.0,5120.0,5120.0
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,...,COMPLETED,cn1473,0,0,65317.33,2721.555417,NaN,122880.0,5120.0,5120.0
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,...,COMPLETED,cn1440,0,0,65876.11,2744.837917,NaN,122880.0,5120.0,5120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,...,COMPLETED,cn432,0,0,23269.08,11634.540000,NaN,32768.0,16384.0,16384.0
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,...,COMPLETED,cn440,0,0,20879.81,10439.905000,NaN,32768.0,16384.0,16384.0
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,...,COMPLETED,cn312,0,0,30042.68,15021.340000,NaN,32768.0,16384.0,16384.0
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,...,COMPLETED,cn312,0,0,31067.75,15533.875000,NaN,32768.0,16384.0,16384.0


In [39]:
pd.set_option('display.max_rows', None)
accre_jobs_2020['nodelist'].value_counts().to_frame()

,nodelist
vm-cms-sam-pri,45761
vm-cms-sam-sec,33390
ng518,20910
cn1091,14308
cn1367,13423
cn304,13233
cn1321,12793
cn449,12142
cn416,12077
cn430,11909


In [40]:
# How to use this function: df["nodelist"].apply(format_nodelist)

def format_nodelist(nodelist):
    """This function takes a list of nodes in any of the pattern format described above and convert it into a tuple in the format (cluster_type, cluster_nodes)
        Both the cluster_type and the cluster_nodes will be strings."""
    
    # We want to return the cluster_type and the cluster_nodes as strings
    cluster_type = "" # e.g. "cn"
    cluster_nodes = "" # e.g. "1308, 134, 545, 546, 547, 548"
    
    # FIRST PART: Handle the cases for:
    #   - Multi-Nodes Cluster List
    #   - Multi-Nodes Cluster Range
    #   - Multi-Nodes Cluster List and Range Combo
    
    # First, remove the "]" and split at the "[" character from the nodelist string
    # If the nodelist string does not contain "[]", this will be ignore and just get the original string
    
    type_and_list = nodelist.strip("]").split("[") # e.g. "cn[1308, 134, 545-548]" => ["cn", "1308, 134, 545-548"]
    
    # If type_and_list has more than one element, we have an actual cluster to process further
    if len(type_and_list) > 1:
        
        # The cluster type is the first element of the list
        cluster_type = type_and_list[0] # "cn"
        
        # The individual nodes can be split on the comma
        nodes = type_and_list[1].split(",") # ["1308", "134", "545-548"]
        
        # The cluster list might need further processing if it has a range
        for nstr in nodes:
            
            # Check if the string contains a dash. If it does, this is a range
            if ("-" in nstr):
                
                # Split on the dash
                range_list = nstr.split("-") # ["545", "548"]
                
                # Convert the range numbers to int
                start_num = int(range_list[0])
                end_num = int(range_list[1])
                
                # Generate the actual range of numbers
                # The first number is the start of the range, second number is the end of the list INCLUSIVE
                range_list_int = [num for num in range(start_num, end_num+1)] # +1 to account for inclusive
                
                # Now, range_list_int == [545, 546, 547, 548]
                # We need to convert it back to a list of string
                range_list_str = list(map(lambda num: "{}".format(num), range_list_int))
                
                # Now, range_list_int == ["545", "546", "547", "548"]
                # Add each one of them to the cluster node
                for n in range_list_str:
                    
                    # Append to the cluster_nodes string
                    cluster_nodes += "," + n
            
            else: # The string does not have a dash so it is just a single node. e.g. "1308" and "134"
                
                # Append directly to the cluster_nodes. Remove white spaces
                cluster_nodes += "," + nstr.strip()
    
    else: # If here, the type_and_list has only one element string
        
        # SECOND PART: Handle the cases for:
        #   - Single Node Cluster
        
        # Use regular expression to find the node_type (cluster_type) and the node_number (cluster_number)
        # Make sure `re` is imported
        
        single_node_cluster_format = r"^(\w{2,3}?)(\d+)" # for cn124, or gpu5738
        
        # Check if the node match the Single Node Cluster pattern
        if re.match(single_node_cluster_format, type_and_list[0]):
            
            # Using re.findall() and Splitting text and number in string
            # This split gives us the node_type and the node_number
            cluster_type, cluster_nodes = re.findall(single_node_cluster_format, type_and_list[0])[0]
        
        else:
            # If we don't get a match, it means it is a Single Node Patternless
            # THIRD PART: Handle the cases for:
            #   - Single Node Patternless
            
            cluster_type = "generic"
            cluster_nodes = type_and_list[0]
    
    # Trim cluster_nodes of any extra commas
    cluster_nodes = cluster_nodes.strip(",")
    
    # Finally, return the cluster_type and the cluster_nodes a a tuple
    return (cluster_type, cluster_nodes)

In [41]:
# Add a column cluster_type_and_nodes
# This will contain the tuple from format_nodelist
accre_jobs_2020 = accre_jobs_2020.assign(cluster_type_and_nodes = accre_jobs_2020['nodelist'].apply(format_nodelist))
accre_jobs_2020.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,...,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn,reqmem_mn_per_core,reqmem_per_core,cluster_type_and_nodes
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,...,cn1531,0,0,65973.49,2748.895417,NaN,122880.0,5120.0,5120.0,"(cn, 1531)"
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,...,cn1441,0,0,67181.12,2799.213333,NaN,122880.0,5120.0,5120.0,"(cn, 1441)"
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,...,cn1464,0,0,69111.86,2879.660833,NaN,122880.0,5120.0,5120.0,"(cn, 1464)"
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,...,cn1473,0,0,65317.33,2721.555417,NaN,122880.0,5120.0,5120.0,"(cn, 1473)"
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,...,cn1440,0,0,65876.11,2744.837917,NaN,122880.0,5120.0,5120.0,"(cn, 1440)"


In [46]:
# Finally, Split the cluster_type_and_nodes column into 2 separate columns
accre_jobs_2020[['cluster_type', 'cluster_nodes']] = pd.DataFrame(accre_jobs_2020['cluster_type_and_nodes'].tolist(), index=accre_jobs_2020.index) 
accre_jobs_2020

In [47]:
# Checking rows with nodes > 150
accre_jobs_2020[accre_jobs_2020["nodes"] > 150]

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,...,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn,reqmem_mn_per_core,reqmem_per_core,cluster_type_and_nodes,cluster_type,cluster_nodes
2781194,22266623,crowns,wilhelmina,5120Mc,54.75M,4-00:00:00,00:01:18,156,1,production,...,0,54.75,0.350962,5120.0,NaN,5120.000,5120.000,"(cn, 313,324,331,332,334,335,336,354,361,365,3...",cn,"313,324,331,332,334,335,336,354,361,365,372,37..."
2781539,22272025,crowns,wilhelmina,5120Mc,9910.64M,4-00:00:00,1-07:49:28,163,1,production,...,0,9910.64,60.801472,5120.0,NaN,5120.000,5120.000,"(cn, 305,313,324,331,332,334,335,336,337,354,3...",cn,"305,313,324,331,332,334,335,336,337,354,361,36..."
3311027,23335998,grisette,acy,12288Mn,2.70M,00:50:00,00:08:53,200,2,production,...,15,2.70,0.006750,NaN,12288.0,30.720,30.720,"(cn, 304,315,316,317,318,319,320,322,323,324,3...",cn,"304,315,316,317,318,319,320,322,323,324,326,32..."
3329704,23352111,grisette,acy,12288Mn,0.46M,00:50:00,00:00:03,200,2,production,...,0,0.46,0.001150,NaN,12288.0,30.720,30.720,"(cn, 315,316,317,318,319,320,322,323,324,326,3...",cn,"315,316,317,318,319,320,322,323,324,326,327,32..."
3343504,23375092,grisette,acy,12288Mn,2.83M,00:50:00,00:00:31,400,2,production,...,0,2.83,0.003538,NaN,12288.0,15.360,15.360,"(cn, 301,302,303,305,306,307,308,309,311,312,3...",cn,"301,302,303,305,306,307,308,309,311,312,313,31..."
3343506,23375094,grisette,acy,12288Mn,0.62M,00:50:00,00:00:24,500,2,production,...,0,0.62,0.000620,NaN,12288.0,12.288,12.288,"(cn, 301,302,303,305,306,307,308,309,311,312,3...",cn,"301,302,303,305,306,307,308,309,311,312,313,31..."


In [48]:
# List of nodes for jobid == 22272025
accre_jobs_2020[accre_jobs_2020["jobid"] == "22272025"]["cluster_nodes"].values

array(['305,313,324,331,332,334,335,336,337,354,361,365,372,373,374,382,386,404,408,437,438,445,447,472,497,911,912,1085,1086,1088,1090,1092,1094,1095,1096,1121,1122,1123,1126,1127,1128,1129,1132,1201,1202,1203,1207,1208,1212,1213,1214,1215,1216,1221,1223,1224,1228,1241,1258,1260,1261,1262,1263,1269,1270,1278,1279,1281,1285,1286,1287,1291,1292,1299,1305,1306,1307,1308,1310,1313,1319,1321,1323,1325,1331,1332,1335,1336,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1349,1350,1351,1352,1353,1354,1358,1359,1364,1367,1368,1369,1370,1376,1377,1380,1384,1385,1386,1387,1388,1389,1391,1392,1394,1395,1396,1397,1398,1420,1421,1423,1424,1425,1426,1427,1432,1433,1434,1435,1436,1437,1438,1442,1443,1451,1452,1472,1492,1494,1495,1497,1498,1499,1504,1506,1508,1516,1521,1524,1530,1531,1534,1535,1536'],
      dtype=object)

In [49]:
# Checking rows with cluster_Type == 'generic'
accre_jobs_2020[accre_jobs_2020["cluster_type"] == "generic"]["cluster_nodes"].value_counts()

vm-cms-sam-pri      45761
vm-cms-sam-sec      33390
eval-dell-01          381
capra1                 83
dougherty1             70
vm-qa-flatearth1        4
rocksteady              1
Name: cluster_nodes, dtype: int64

In [50]:
accre_jobs_2020['partition'].value_counts()

production              3308929
nogpfs                   327440
sam                       79151
pascal                    48000
turing                    39377
debug                      6738
maxwell                    3347
cgw-capra1                   83
cgw-dougherty1               70
cgw-cqs1                     34
cgw-cqs3                      4
cgw-vm-qa-flatearth1          4
devel                         2
cgw-tbi01                     2
cgw-rocksteady                1
Name: partition, dtype: int64

In [52]:
production_nodes = accre_jobs_2020['partition'] == 'production'
production_nodes

KeyboardInterrupt: 

In [53]:
accre_jobs_2020.to_csv(index=False)

In [56]:
accre_jobs_2020.to_csv('C:/Users/Owner/nssds4/dq2-accre-amd/accre_jobs_cleaned.csv', index = False)